In [19]:
#Import pulp and Pandas
from pulp import *
import pandas as pd
import xlrd

In [20]:
#test Pulp
pulp.pulpTestAll()

	 Testing zero subtraction
	 Testing inconsistant lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing Long lines in LP
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing column based modelling
	 Testing dual variables and slacks reporting
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)
* Solver <class 'pulp.solvers.PULP_CBC_CMD'> passed.
Solver <class 'pulp.solvers.CPLEX_DLL'> un

In [21]:
#read in data
data = pd.read_excel('diet.xls', 
                      skip_footer=3)
data

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
5,"Peppers, Sweet, Raw",0.53,1 Pepper,20.0,0.0,0.1,1.5,4.8,1.3,0.7,467.7,66.1,6.7,0.3
6,"Potatoes, Baked",0.06,1/2 Cup,171.5,0.0,0.2,15.2,39.9,3.2,3.7,0.0,15.6,22.7,4.3
7,Tofu,0.31,1/4 block,88.2,0.0,5.5,8.1,2.2,1.4,9.4,98.6,0.1,121.8,6.2
8,Roasted Chicken,0.84,1 lb chicken,277.4,129.9,10.8,125.6,0.0,0.0,42.2,77.4,0.0,21.9,1.8
9,Spaghetti W/ Sauce,0.78,1 1/2 Cup,358.2,0.0,12.3,1237.1,58.3,11.6,8.2,3055.2,27.9,80.2,2.3


In [22]:
#convert to list
data = data.values.tolist()

In [23]:
#read in requirements
requirements = pd.read_excel('diet.xls', 
                             skiprows=list(range(1,66)),
                             usecols=[2,3,4,5,6,7,8,9,10,11,12,13],
                             header=0, 
                             index_col=0)

In [24]:
requirements

,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
Serving Size,,,,,,,,,,,
Minimum daily intake,1500,30,20,800,130,125,60,1000,400,700,10
Maximum daily intake,2500,240,70,2000,450,250,100,10000,5000,1500,40


In [25]:
# Use .loc to get specific requirements when building model
# Ex. 'Calories' and 'Minimum daily intake'
requirements.loc['Minimum daily intake','Calories']

1500

In [26]:
requirements.loc['Maximum daily intake','Calories']

2500

In [27]:
#create list of foods, then dictionaries for each nutrient with food as key and amount of nutrient as value

foods = [x[0] for x in data]
cost = dict([(x[0], float(x[1])) for x in data])
calories = dict([(x[0], float(x[3])) for x in data])
cholesterol = dict([(x[0], float(x[4])) for x in data])
fat = dict([(x[0], float(x[5])) for x in data])
sodium = dict([(x[0], float(x[6])) for x in data])
carbs = dict([(x[0], float(x[7])) for x in data])
fiber = dict([(x[0], float(x[8])) for x in data])
protein = dict([(x[0], float(x[9])) for x in data])
vita = dict([(x[0], float(x[10])) for x in data])
vitc = dict([(x[0], float(x[11])) for x in data])
calcium = dict([(x[0], float(x[12])) for x in data])
iron = dict([(x[0], float(x[13])) for x in data])

In [45]:
# Initialize Pulp Optimization Object
diet = LpProblem('diet', LpMinimize)

In [46]:
#Create food variables
foodvars = LpVariable.dict('Foods', foods, 0)

In [47]:
#create binary variables
chosenvars = LpVariable.dicts("Chosen", foods, 0, 1, 'Binary')

In [48]:
#Objective Function
diet += lpSum([cost[f] * foodvars[f] for f in foods]), 'Total Cost'

In [49]:
# add in additional constraints for max and min nutrients
diet += lpSum([calories[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Calories'], 'min calories'
diet += lpSum([calories[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Calories'], 'max calories'

In [50]:
diet += lpSum([cholesterol[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Cholesterol mg']
diet += lpSum([cholesterol[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Cholesterol mg']

In [51]:
diet += lpSum([fat[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Total_Fat g']
diet += lpSum([fat[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Total_Fat g']

In [52]:
diet += lpSum([sodium[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Sodium mg']
diet += lpSum([sodium[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Sodium mg']

In [53]:
diet += lpSum([carbs[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Carbohydrates g']
diet += lpSum([carbs[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Carbohydrates g']

In [54]:
diet += lpSum([fiber[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Dietary_Fiber g']
diet += lpSum([fiber[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Dietary_Fiber g']

In [55]:
diet += lpSum([protein[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Protein g']
diet += lpSum([protein[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Protein g']

In [56]:
diet += lpSum([vita[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Vit_A IU']
diet += lpSum([vita[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Vit_A IU']

In [57]:
diet += lpSum([vitc[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Vit_C IU']
diet += lpSum([vitc[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Vit_C IU']

In [58]:
diet += lpSum([calcium[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Calcium mg']
diet += lpSum([calcium[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Calcium mg']

In [59]:
diet += lpSum([iron[f] * foodvars[f] for f in foods]) >= requirements.loc['Minimum daily intake','Iron mg']
diet += lpSum([iron[f] * foodvars[f] for f in foods]) <= requirements.loc['Maximum daily intake','Iron mg']

In [60]:
# Add constraint so that at least .1 servings of a food are required if it is selected. 
for f in foods:
    diet += foodvars[f] >= .1*chosenvars[f]
# and a constraint for max of any food   
for f in foods:
    diet += foodvars[f] <= 999999999*chosenvars[f]

In [61]:
# Add constraint so that celery and brocolli both aren't selected
diet += chosenvars['Frozen Broccoli'] + chosenvars['Celery, Raw'] <= 1, 'Broc/Celery Req'

In [62]:
# Add constraint so that at least 3 proteins are selected
diet += chosenvars['Roasted Chicken'] + chosenvars['White Tuna in Water'] + chosenvars['Poached Eggs'] + chosenvars['Scrambled Eggs'] + chosenvars['Bologna,Turkey'] + chosenvars['Frankfurter, Beef'] + chosenvars['Ham,Sliced,Extralean'] + chosenvars['Kielbasa,Prk'] + chosenvars['Hamburger W/Toppings'] + chosenvars['Hotdog, Plain'] + chosenvars['Taco'] + chosenvars['Pork'] + chosenvars['White Tuna in Water'] + chosenvars['Sardines in Oil'] + chosenvars['Chicknoodl Soup'] + chosenvars['Splt Pea&Hamsoup']  + chosenvars['Vegetbeef Soup'] + chosenvars['Neweng Clamchwd'] + chosenvars['Beanbacn Soup,W/Watr'] >= 3, 'Protein Req'


In [63]:
#Solve and check solution
diet.solve()
LpStatus[diet.status]

'Optimal'

In [64]:
#Print foods and their amounts for an optimal diet
for v in diet.variables():
    if v.varValue>0:
        print (v.name, "=", v.varValue)

Chosen_Celery,_Raw = 1.0
Chosen_Kielbasa,Prk = 1.0
Chosen_Lettuce,Iceberg,Raw = 1.0
Chosen_Oranges = 1.0
Chosen_Peanut_Butter = 1.0
Chosen_Poached_Eggs = 1.0
Chosen_Popcorn,Air_Popped = 1.0
Chosen_Scrambled_Eggs = 1.0
Foods_Celery,_Raw = 42.399358
Foods_Kielbasa,Prk = 0.1
Foods_Lettuce,Iceberg,Raw = 82.802586
Foods_Oranges = 3.0771841
Foods_Peanut_Butter = 1.9429716
Foods_Poached_Eggs = 0.1
Foods_Popcorn,Air_Popped = 13.223294
Foods_Scrambled_Eggs = 0.1
